In [44]:
#Import my dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [45]:
#Read in the cleaned up training/testing csv, also use auth4_train to inspect further
auth4_train = pd.read_csv("auth4_training.csv", nrows=50000)
auth4_train.head()
#auth4_validate.head()

,Unnamed: 0,time,dst_user,src_comp,dst_comp,auth_type,logon_type,auth_orientation,succ_fail
0,0,726131,U9040,C19193,C625,Kerberos,Network,LogOn,Success
1,1,726131,U9093,C19290,C529,Kerberos,Network,LogOn,Success
2,2,726131,U9093,C19290,C586,Kerberos,Network,LogOn,Success
3,3,726131,U9136,C19359,C5618,NTLM,Network,LogOn,Success
4,4,726131,U9292,C19632,C2162,Kerberos,Network,LogOn,Success


In [46]:
#drop the Unnamed index that was exported in Parse_Data2 notebook
auth4_train1 = auth4_train.drop(columns=['Unnamed: 0'])


In [47]:
#Converts succ_fail to numeric where Success equals 1, fail equals 0
auth4_train2 = pd.get_dummies(auth4_train1, columns=['succ_fail'], drop_first=True)
auth4_train2.head()

,time,dst_user,src_comp,dst_comp,auth_type,logon_type,auth_orientation,succ_fail_Success
0,726131,U9040,C19193,C625,Kerberos,Network,LogOn,1
1,726131,U9093,C19290,C529,Kerberos,Network,LogOn,1
2,726131,U9093,C19290,C586,Kerberos,Network,LogOn,1
3,726131,U9136,C19359,C5618,NTLM,Network,LogOn,1
4,726131,U9292,C19632,C2162,Kerberos,Network,LogOn,1


In [48]:
target = auth4_train2['succ_fail_Success']
target_names = ["Success", "Failure"]
auth4_train3 = auth4_train2.drop(columns=['auth_orientation', 'succ_fail_Success'])

In [49]:
#Get dummies to convert categorical to numeric
data = auth4_train3
data.head()

,time,dst_user,src_comp,dst_comp,auth_type,logon_type
0,726131,U9040,C19193,C625,Kerberos,Network
1,726131,U9093,C19290,C529,Kerberos,Network
2,726131,U9093,C19290,C586,Kerberos,Network
3,726131,U9136,C19359,C5618,NTLM,Network
4,726131,U9292,C19632,C2162,Kerberos,Network


In [50]:
X = data
y = target

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
enc = OneHotEncoder(handle_unknown='ignore')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [52]:
#Set up variables for use below
import copy
X_train_rare = copy.copy(X_train)
X_test_rare = copy.copy(X_test)
X_train_rare["test"]=0
X_test_rare["test"]=1
les = []

#
temp_df = pd.concat([X_train_rare,X_test_rare],axis=0)
names = list(X_train_rare.columns.values)
temp_df = pd.concat([X_train_rare,X_test_rare],axis=0)
for i in names:
    temp_df.loc[temp_df[i].value_counts()[temp_df[i]].values < 20, i] = "RARE_VALUE"
for i in range(temp_df.shape[1]):
    temp_df.iloc[:,i]=temp_df.iloc[:,i].astype('str')
    #Below is where you pull back out your X_train and X_test
X_train_rare = temp_df[temp_df["test"]=="0"].iloc[:,:-1].values
X_test_rare = temp_df[temp_df["test"]=="1"].iloc[:,:-1].values
#Apply preprocessing to X_train
for i in range(X_train_rare.shape[1]):
    le = preprocessing.LabelEncoder()
    le.fit(temp_df.iloc[:,:-1].iloc[:, i])
    les.append(le)
    X_train_rare[:, i] = le.transform(X_train_rare[:, i])
    X_test_rare[:, i] = le.transform(X_test_rare[:, i])
enc.fit(X_train_rare)
X_train_rare = enc.transform(X_train_rare)
X_test_rare = enc.transform(X_test_rare)
X_train_rare

<37500x1078 sparse matrix of type '<class 'numpy.float64'>'
	with 225000 stored elements in Compressed Sparse Row format>

In [53]:
from sklearn.neighbors import KNeighborsClassifier 
clf = KNeighborsClassifier(n_neighbors=2)

In [55]:

clf.fit(X_train_rare, y_train)
y_pred = clf.predict_proba(X_test_rare)
y_pred

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [56]:
from sklearn.metrics import log_loss
print(log_loss(y_test,y_pred))

0.06472532244062427


In [57]:
print("Test set predictions: {}".format(clf.predict(X_test_rare)))


Test set predictions: [1 1 1 ... 1 1 1]


In [58]:
print("Test set accuracy: {: .2f}".format(clf.score(X_test_rare, y_test)))

Test set accuracy:  0.99
